# Predictive Maintenance of Turbofan Engines

## SageMaker MXNet Estimator

First we'll import our variables from the pervious notebook



In [ ]:
import pickle

with open('shared_vars', 'rb') as f:
    bucket, prefix, s3_train_data = pickle.load(f)

### MXNet Model Training Script (Out of scope)

Training MXNet models using MXNet Estimators is a two-step process. First, you prepare your training script, then second, you run this on SageMaker via an MXNet Estimator. The training script we have prepared for the model is located in the entry_point folder.

The training script contains functions to create the model for training and for inference. We also have functions to convert our dataframes into a Gluon Dataset so that it can be efficiently prefetched, transformed into numerical features used by the network and padded so that we can learn from multiple samples in batches.

For more information on how to setup a training script for SageMaker using the MXNet estimator see: https://sagemaker.readthedocs.io/en/stable/using_mxnet.html#preparing-the-mxnet-training-script

**Important note**

** The upper bound for the RUL is set to 130 in the training script, this means that the predictions from the model will be a fraction of this value **


In [ ]:
import numpy as np
import pandas as pd
import sagemaker
import os
import io
import json
import boto3
from time import strftime, gmtime

from sagemaker import get_execution_role
from sagemaker.mxnet import MXNet

role = get_execution_role()

We import the training data that we previously saved to CSV.

In [ ]:
train_df = []

for i in range(0,4):
    df = pd.read_csv('data/train-{:01d}.csv'.format(i), index_col=0)
    train_df.append(df)

### Train MXNet Estimator

We now start the Sagemaker training job by creating an MXNet estimator. We pass some arguments to the MXNet estimator constructor such as `entry_point`, `instance_count` and `instance_type`.


In [ ]:
model_name = "pred-maintenance-mxnet-model"
training_job_name = "{}-{}".format(model_name, strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
train_instance_type = 'ml.p3.2xlarge'
output_location = 's3://{}/{}/output'.format(bucket, prefix)

m = MXNet(entry_point='script.py',
          source_dir='entry_point',
          py_version='py3',
          role=role, 
          instance_count=1, 
          instance_type=train_instance_type,
          output_path=output_location,
          hyperparameters={'num-datasets' : len(train_df),
                           'num-gpus': 1,
                           'epochs': 500,
                           'optimizer': 'adam',
                           'batch-size':1,
                           'log-interval': 100},
         input_mode='File',
         use_spot_instances = True,
         max_run = 3600,
         max_wait = 3600,
         framework_version='1.6.0')

We kick off the trianing job by calling the fit method. fit has a required argument of the S3 training data location, and we also pass an optional job_name argument which we will use later to call the model for batch transformation.

In [ ]:
m.fit({'train': s3_train_data}, job_name=training_job_name)

## Deploy the model


### Create Transformer Model
We now call the transformer function which will take the training model and create a SageMaker model suitable for deployment.

In [ ]:
batch_output = 's3://{}/{}/{}'.format(bucket, prefix, 'batch-inference')
transformer = m.transformer(instance_count=1, instance_type='ml.m4.xlarge', output_path=batch_output)

### Batch transform of test data using the transformer model

Using the `transformer` model that we just created we can run a SageMaker Batch Transformation job to get some predictions on the test data sets that we have.

Below is a function that copies some test data to a new location in S3 where it's then used as the input for the `transform` fucntion for the SageMaker Batch Transformation Job.

In [ ]:
s3_test_key = '{}/data/test-0.csv'.format(prefix)
s3_transform_input = os.path.join(prefix,  "batch-transform-input")

def get_transform_input():
    s3_client = boto3.client('s3')
    s3_response = s3_client.get_object(Bucket=bucket, Key=s3_test_key)
    test_file = s3_response["Body"].read()

    test_df_entry = pd.read_csv(io.BytesIO(test_file))
    test_data = test_df_entry[test_df_entry['id']==0+1][test_df_entry.columns[2:-1]].values
    test_data = test_data[0:test_data.shape[0]-1,:].astype('float32')
    data_payload = {'input':np.expand_dims(test_data, axis=0).tolist()}
    
    job_name = 'predictive-maintenance-batch-transform-job-{}'.format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
    s3_batch_transform_input_key = os.path.join(s3_transform_input, job_name)
    
    s3_client.put_object(Body=json.dumps(data_payload),
                         Bucket=bucket, 
                         Key=s3_batch_transform_input_key)
    return job_name, 's3://{}/{}'.format(bucket, s3_batch_transform_input_key)

job_name, input_key = get_transform_input()
transformer.transform(input_key, wait=True)

### View prediction results

Once the SageMaker Batch Transform job completes we can see the prediction of the fractional remaining useful life for the sensor readings we provided.

In [ ]:
def get_transform_output():
    s3_client = boto3.client('s3')
    s3_response = s3_client.get_object(Bucket=bucket, Key=os.path.join(prefix, 
                                                                       'batch-inference', 
                                                                       job_name+'.out'))
    transform_out = np.array(eval(s3_response["Body"].read()))
    return transform_out
    
transform_output = get_transform_output()

In [ ]:
print(transform_output)

To see the actual number of predicted cycles until failure we simply multiply the output by our upper bound that was set in the traiing script, 130 in this case.

In [ ]:
print(transform_output * 130)